# Simple Vanilla RNN Implementation

Build vanilla RNN using Tensorflow that takes a binary sequence X and predict a binary sequence Y.

## Data
Input X and output Y are binary sequences.
### Input Sequence (X)
* 50% chance of X<sub>t</sub> being 1 at time step *t*
### Output Sequence (Y)
* Each time step *t*:
    * Y<sub>t</sub> has a 50% *base* chance of being 1
    * If X<sub>t-3</sub>=1, Y<sub>t</sub> chance of being 1 increases 50% (i.e to 100%)
    * If X<sub>t-8</sub>=1, Y<sub>t</sub> chance of being 1 decreases 25% (i.e to 25%)
    * If X<sub>t-3</sub>=1 **and** X<sub>t-8</sub>=1, Y<sub>t</sub> chance of being 1 = 50%+50%-25%=75%

## Model Architecture
At each time step *t*:
* Inputs: a **one-hot** binary X<sub>t</sub> vector and a previous state vector S<sub>t-1</sub>
* Outputs: State vector S<sub>t</sub> and a predicted probability distributino vector P<sub>t</sub> for then one-hot binary vector Y<sub>t</sub> 

### Formal Definitions
* S<sub>t</sub> = tanh(W(X<sub>t</sub>@S<sub>t-1</sub>)+b<sub>s</sub>)
* P<sub>t</sub> = softmax(US<sub>t</sub> + b<sub>p</sub>)

Where:
* @ represents vector *concatenation*
* X<sub>t</sub> is a one-hot binary vector

## Tensorflow Graph
* *n* units (time steps) wide
    * Truncated Backpropagation
    * Split data into size *n* sequences and feed through network separately
* Each unit is a duplicate -> sharing same variables
* Represent duplicate tensors as a **list of tensors**
    * rnn inputs, rnn outputs (hidden state), predictions and loss
    
![rnn_architecture](./jupyter-assets/BasicRNNLabeled.png)

## Code Outline
* Generate Sequence Data
    * Produce X and Y arrays according to the rules above
* Batch the data
    * Set a `batch_size`
    * Split X and Y into `batch_size` length sequences:
        * Vertically stacked data matrix
        * Shape example:
        ```Python
        X = [1,2,3,4,5,6,7,8,9,10]
        batch_size = 2
        # 5 batches, size two
        X_batches = [[1,2,3,4,5],
                     [6,7,8,9,10],]
        # first batch = [1, 6], second batch = [2,7] etc
        
        ```
    * Further partition into `n` units for truncated backpropagation
    
* Create Model
* Train Model

In [2]:
import numpy as np
import tensorflow as tf

batch_size = 200
truncate_steps = 7


In [36]:
def gen_data(size=1000000):
    X = np.random.choice(2, size=(size,))
    Y = []
    for i in range(size):
        p = 0.5
        if X[i-3] == 1:
            p += 0.5
        if X[i-8] == 1:
            p -= 0.25
        if np.random.rand() > p:
            Y.append(1)
        else:
            Y.append(0)
    return X, np.array(Y)
data = gen_data()

In [44]:
def batch_data_gen(data, batch_size, truncate_steps):
    X, Y = data
    data_len = len(X)

    num_batches = data_len // batch_size
    X_batch = np.zeros([batch_size, num_batches], dtype=np.int32)
    Y_batch = np.zeros([batch_size, num_batches], dtype=np.int32)
    for i in range(batch_size):
        X_batch[i] = X[num_batches * i: num_batches * (i+1)]
        Y_batch[i] = X[num_batches * i: num_batches * (i+1)]

    truncate_size = num_batches // truncate_steps

    for i in range(truncate_size):
        x = X_batch[:, truncate_steps * i: truncate_steps * (i+1)]
        y = Y_batch[:, truncate_steps * i: truncate_steps * (i+1)]
        yield(x, y)


def epoch_data_gen(n, truncate_steps):
    for i in range(n):
        yield batch_data_gen(generate_data(), batch_size, truncate_steps)

(200, 5000)
